# 训练音乐推荐系统

基于现有的 Word2vec 模型。

把每张播放列表中的曲目 id 和在一起，作为一个个的句子，喂给 w2v：

```py
sentences = [
    ["track_1_id", "track_2_id", ...], # playlist_1
    [...], # playlist_2
    ...
]
```

In [1]:
# 这个 cell 没用，草稿，我先试一下怎么获取训练的语料

import sqlite3

conn = sqlite3.connect('/Volumes/shared/murecom/intro/spotify/playlists.db')
cp = conn.cursor()
cp.execute("select id from playlists")

# pid = cp.fetchone()[0]
# print(pid)
cp.fetchone()

ct = conn.cursor()
# ct.execute("select track_id from playlist_tracks where playlist_id=?", (pid,))
# ct.fetchall()
p = cp.fetchone()
while p is not None:
    pid = p[0]

    ct.execute("select track_id from playlist_tracks where playlist_id=?", (pid,))
    # sentences.append(ct.fetchall())
    tracks = ct.fetchall()
    tracks = [t[0] for t in tracks]
    print(len(tracks), tracks)

    # next playlist
    p = cp.fetchone()
    break
ct.close()
cp.close()
conn.close()

39 ['01apQz7E72krU1k1b4VWs7', '0I4GET1to4HFDGOD7C2yZq', '0KDn1sQyUdgD4X1LYlEbTe', '0gthY07uquYTZXfg1Kc5EX', '0nHQAxZgzAcKGRxAsOISTc', '0trTpNYmHAK3EihaZBd2h6', '0uxPr0jgZcSu2E2CbazF1q', '180MVaidIYWpB1MSYEv5BI', '1EbUvGMGNtR5VoJOXDwgm6', '1LGv7Ah6TXp1soAAIzzuGC', '1k6GEdfONk9Fs6JgrC8aBN', '2FppDwkXcO6AveFobwZGS0', '2QnRyMZ7nCQdopDlwef4lm', '2mqOYFXvkB2N9B4zDKHlgA', '2q4PnncxGIm1oVQzu6hS2d', '2yD6IiTv51oeFY3pC4cOLj', '41L3O37CECZt3N7ziG2z7l', '4DG4um6R0wx4WrqhhACc5g', '4Xn2RsLiDUDisOgJ24FigK', '4bw49rU057FSUTpGF4pnxq', '50bWipKseUhTCP5YPiGRbA', '55hbMmAsueVEBTdwbv3TiF', '57G7JxlRQ40mdLFKjI8PWo', '5HAdd9vei9veddPVJXcG7F', '5NF2n5zIR3t2P6p44ieyI8', '5Ush5YElzq1zjldlllP7hp', '5ZrDlcxIDZyjOzHdYW1ydr', '5ygDXis42ncn6kYG14lEVG', '65ezkm2CfwVsYVhaymLrUB', '6BqdNDLZ3Pdcly46pu6nwj', '6IDh95GKLucPusCHJJvLLD', '6R15sXb3qRfJOsIbt1vS2a', '6iSpyAN4PjyCnxcJthU4Jl', '6iXBQXYVSMSY7LsZ7lj161', '6jF6iozvmHms55tWmsXMud', '6riYDJXW8ql7LTW4Wb9S7V', '6yH3xaGkj9OHFPCk7JiVPc', '71fp15098CPhaPyQ8t7ldo', '7dlNLDr

In [2]:
import sqlite3


def playlist_tracks_gen(db: str):
    """从数据库 db 中获取每张播放列表的曲目id

    :return: 生成器，yield [tid, tid, ...] of a playlist
    """
    conn = sqlite3.connect(db)
    try:
        cp = conn.cursor()
        cp.execute("select id from playlists")

        p = cp.fetchone()
        while p is not None:
            pid = p[0]
            # fetch tracks in playlist p
            ct = conn.cursor()
            ct.execute("select track_id from playlist_tracks where playlist_id=?", (pid,))
            tracks = ct.fetchall()
            ct.close()

            # [("",), ("id",), ("id", ), ...] -> ["id", "id", ...]
            tracks = [t[0] for t in tracks if t and t[0]]
            yield tracks

            # next playlist
            p = cp.fetchone()
        cp.close()
    except:
        raise
    finally:
        conn.close()


In [3]:
# 试一下生成器
sentences = playlist_tracks_gen("/Volumes/shared/murecom/intro/spotify/playlists.db")

i = 0
for s in sentences:
    print(len(s), s[0])
    i += 1
    if i > 3:
        break

737 01JszWYuKqRjddPQ4creVF
39 01apQz7E72krU1k1b4VWs7
46 0A1hoCfMLkiAgvhWkkucJa
73 0Hja9zlVQHC768PsPjWscW


In [4]:
class PlaylistTracksIter(object):
    def __init__(self, db):
        self.s = playlist_tracks_gen(db)

    def __iter__(self):
        yield from self.s


DB = '/Volumes/shared/murecom/intro/spotify/playlists.db'

In [5]:
# 试一下迭代器
i = 0
for s in PlaylistTracksIter(DB):
    print(len(s), s[0])
    i += 1
    if i > 3:
        break

737 01JszWYuKqRjddPQ4creVF
39 01apQz7E72krU1k1b4VWs7
46 0A1hoCfMLkiAgvhWkkucJa
73 0Hja9zlVQHC768PsPjWscW


In [6]:
import gensim

MODEL_FILE = '/Volumes/shared/murecom/intro/songs.word2vec'

# Train a new model
# model = gensim.models.Word2Vec(sentences=PlaylistTracksIter(DB), min_count=4)
# with open(MODEL_FILE, "wb") as f:
#     model.save(f)

model = gensim.models.Word2Vec.load(MODEL_FILE)

In [7]:
model.wv['01JszWYuKqRjddPQ4creVF']

array([ 4.5684148e-03,  5.2385400e-03, -7.7299788e-03,  1.5946460e-03,
       -5.0252248e-03, -1.4793158e-03,  8.8905236e-03,  1.2116050e-03,
        7.0763850e-03,  7.2929380e-04, -1.6901898e-03,  6.6553974e-03,
        5.4897307e-03,  1.5793634e-03, -3.3323001e-03, -6.2783505e-03,
       -6.6507314e-03, -8.8431053e-03,  1.3390756e-03,  1.3452291e-03,
        6.2577580e-03,  6.3973619e-03, -3.6780667e-03,  5.4709911e-03,
        7.5929998e-03, -6.0517122e-03,  5.0346851e-03, -7.9479171e-03,
       -9.5174601e-03,  6.5671682e-04,  8.9104269e-03, -3.2153679e-03,
        7.9768253e-03,  6.3682557e-03, -6.2449123e-03, -5.8733011e-03,
        6.1626290e-03, -4.9687121e-03, -5.9139179e-03, -3.4214591e-03,
       -6.5741013e-03, -1.9755459e-03, -3.9628460e-03, -6.6205263e-03,
        3.4593535e-03,  2.0494533e-03,  2.8128552e-03,  3.9031242e-03,
        1.0645962e-03, -5.6292294e-03,  5.2893758e-03,  2.8771926e-03,
        4.9405717e-03,  2.3548030e-03, -7.6599433e-03,  6.3430406e-03,
      

In [8]:
model.wv.get_vecattr("01JszWYuKqRjddPQ4creVF", "count")

39

In [9]:
import sqlite3
from operator import itemgetter

conn = sqlite3.connect(DB)

def find_artists(track_id: str) -> list:
    ca = conn.cursor()  # get artists
    ca.execute(
        "select a.name from artists a inner join track_artists ta on a.id = ta.artist_id where ta.track_id=?",
        (track_id,))
    a = [a[0] for a in ca.fetchall()]
    ca.close()
    return a

def find_song(song_name: str, limit=10):
    c = conn.cursor()

    # 用 FTS 优化了: https://www.sqlite.org/fts5.html
    #   CREATE VIRTUAL TABLE tracks_name_fts USING fts5(name, id);
    #   INSERT INTO tracks_name_fts SELECT name, id FROM tracks;
    c.execute(f"SELECT id, name FROM tracks_name_fts WHERE name MATCH '{song_name}' LIMIT {limit}")

    def tracks_id(t):
        return t[0]
    def tracks_name(t):
        return t[1]

    res = sorted((x + (model.wv.get_vecattr(tracks_id(x), "count"),)
                  for x in c.fetchall() if tracks_id(x) in model.wv),
                 key=itemgetter(-1), reverse=True)
    res = [*res][:limit]

    ca = conn.cursor()  # get artists
    for i in range(len(res)):
        tid = tracks_id(res[i])
        a = find_artists(tid)
        res[i] = list(res[i])
        res[i].append(a)
    ca.close()
    c.close()

    return [{"id": tracks_id(r), "name": tracks_name(r), "artists": r[-1], "count": r[-2]} for r in res]

In [10]:
for t in find_song('the sound of silence'):
    # print(t[4], t[5], t[-1], t[-2])  # id, name, artists, count_in_wv
    print(t)

{'id': '1Cj2vqUwlJVG27gJrun92y', 'name': 'The Sound of Silence', 'artists': ['Disturbed'], 'count': 486}
{'id': '5y788ya4NvwhBznoDIcXwK', 'name': 'The Sound of Silence - Acoustic Version', 'artists': ['Simon & Garfunkel'], 'count': 340}
{'id': '2LkaNhCrNVmcYgXJeLVmsw', 'name': 'The Sound of Silence', 'artists': ['Simon & Garfunkel'], 'count': 280}
{'id': '0eZBeB2xFIS65jQHerispi', 'name': 'The Sound of Silence', 'artists': ['Disturbed'], 'count': 263}
{'id': '64gpgh5jBMxyh7iVJll23t', 'name': 'The Sound of Silence', 'artists': ['Simon & Garfunkel'], 'count': 99}
{'id': '2YplrdHMBoRdnHgMeHEwHm', 'name': 'The Sound of Silence - Electric Version', 'artists': ['Simon & Garfunkel'], 'count': 56}
{'id': '0IHE30guMa90mZetZHVbuf', 'name': 'The Sound of Silence', 'artists': ['Joseph Sullinger'], 'count': 20}
{'id': '5ye2aEP0ICWJgJCsawlCp3', 'name': 'The Sound of Silence - Overdubbed Version', 'artists': ['Simon & Garfunkel'], 'count': 6}
{'id': '5N35prOM01qSay3vZyoTgU', 'name': 'É por você que ca

In [11]:
def suggest_songs(song_id):
    similar = dict(model.wv.most_similar([song_id]))
    song_ids = ', '.join(("'%s'" % x) for x in similar.keys())

    c = conn.cursor()
    c.execute("SELECT * FROM tracks WHERE id in (%s)" % song_ids)

    res = sorted((rec + (similar[rec[4]], find_artists(rec[4])) for rec in c.fetchall()),
                 key=itemgetter(-1),
                 reverse=True)
    res = [*res]

    return [{"id": r[4], "name": r[5], "artists": r[-1], "similar": r[-2]} for r in res]


In [12]:
for t in suggest_songs('5y788ya4NvwhBznoDIcXwK'):
    print(t)

{'id': '2NnrAdjE9cPdMklonMBuAv', 'name': 'willow - dancing witch version (Elvira remix)', 'artists': ['Taylor Swift', 'ELVIRA'], 'similar': 0.5026569962501526}
{'id': '44dMahNEJeTOgOB26HdEcR', 'name': 'I Knew You Were Trouble.', 'artists': ['Taylor Swift'], 'similar': 0.4139394164085388}
{'id': '4Z5AzZzTu01gzxZ7yCCO4C', 'name': 'Take One Step', 'artists': ['Tatiana Manaois'], 'similar': 0.41184818744659424}
{'id': '06CRvgRb4fKQXpl2WPf2Cy', 'name': 'Meridian (Capo Song)', 'artists': ['TOPAZ'], 'similar': 0.4183271527290344}
{'id': '4IePM62thd8OrAg8ELrYzO', 'name': 'Aléjate de Mi', 'artists': ['Sonido Profesional'], 'similar': 0.46160659193992615}
{'id': '0qJeyYAgv6UpvewUxRXAhb', 'name': 'Rags2Riches 2 (feat. Lil Baby)', 'artists': ['Rod Wave', 'Lil Baby'], 'similar': 0.42182764410972595}
{'id': '7FBMtFzffz4VIXyA9yAAwA', 'name': 'Hijo de la Luna', 'artists': ['Leo Rojas'], 'similar': 0.4233776926994324}
{'id': '3TDaJHpLfApGdlRNEuNrD2', 'name': 'Ants Marching - Live at Piazza Napoleone, L

In [13]:
def suggest_from(song_name: str):
    s = find_song(song_name, limit=1)
    return s + suggest_songs(s[0]["id"])

In [14]:
suggest_from("easy on me")

[{'id': '0gplL1WMoJ6iYaPgMCL0gX',
  'name': 'Easy On Me',
  'artists': ['Adele'],
  'count': 3156},
 {'id': '2PAoeEUqDSuhvzPv0Hr6yd',
  'name': 'AFTERGLOW',
  'artists': ['YTD', 'Sixthells'],
  'similar': 0.46074774861335754},
 {'id': '10Nn7Qus3AwpOK5R2BuYLe',
  'name': 'Insomnia Relief with Rain Sounds',
  'artists': ['The Earbookers'],
  'similar': 0.419079065322876},
 {'id': '6c9ZDwo8E81PpZGoWzc9VF',
  'name': 'Horseplay',
  'artists': ['Sada Baby'],
  'similar': 0.4128553867340088},
 {'id': '3i7ZOuSmoKYSgLQuohMcig',
  'name': 'Good Good Father',
  'artists': ['Pat Barrett', 'Chris Tomlin'],
  'similar': 0.42512083053588867},
 {'id': '20KXOKumUNxisRWv74hvUF',
  'name': 'Celsius',
  'artists': ['PNL'],
  'similar': 0.4286656975746155},
 {'id': '4iNL0ZOUDAuxkKvE8mEY3p',
  'name': "Let's Talk",
  'artists': ['Josh Dally', 'Timecop1983'],
  'similar': 0.4052392840385437},
 {'id': '5skQzF4pHBxhDogXslWN2X',
  'name': 'Göran Soul Share',
  'artists': ['Jaded Satire', 'Chill Select'],
  'si

In [16]:
suggest_from("Izzo")

[{'id': '10EGxvr4fSE42ABWZeJ3K3',
  'name': 'Izzo (H.O.V.A.)',
  'artists': ['JAY-Z'],
  'count': 34},
 {'id': '56sAXFMfTagQGl5KOxvdxt',
  'name': 'I Wanna Do It All',
  'artists': ['Terri Clark'],
  'similar': 0.421507328748703},
 {'id': '6UsZZfgK2QLinkxCsk4P2s',
  'name': 'Kaputt wie ich',
  'artists': ['Tarek K.I.Z'],
  'similar': 0.43599334359169006},
 {'id': '7a7nlgfjoeEtIAW0QJzU4m',
  'name': 'Nerve',
  'artists': ['Soilwork'],
  'similar': 0.4187915027141571},
 {'id': '1of4TwihFCtwaMSlgkFHLN',
  'name': 'Soodamani',
  'artists': ['Pop Shalini', 'Ranjith Govind'],
  'similar': 0.4234713912010193},
 {'id': '6ayleuWDwuBXJ7LiTJiZnC',
  'name': "Walkin' in LA Freestyle",
  'artists': ['Lil Double 0'],
  'similar': 0.42755126953125},
 {'id': '1T8YONUfkmHPqmUueSYq0V',
  'name': 'Before Meaning Comes',
  'artists': ['Ian William Craig'],
  'similar': 0.4512949287891388},
 {'id': '5tQvGagX5oeTbPL6sXsiNy',
  'name': 'Sugar',
  'artists': ['Guitar Tribute Players'],
  'similar': 0.42138871

In [17]:
suggest_from("my name is")

[{'id': '4rc9kQ6qyssElIccFJgbJb',
  'name': 'My Name Is',
  'artists': ['Eminem'],
  'count': 119},
 {'id': '3q0wgJXeUkhq2jeZ9JS7KN',
  'name': "I'm on Fire",
  'artists': ["Whitey Morgan and the 78's"],
  'similar': 0.4211234450340271},
 {'id': '5XWplGAAmRPPGGbMpMnzy4',
  'name': 'History - Live',
  'artists': ['UPPERROOM'],
  'similar': 0.445537805557251},
 {'id': '0rY0wiJbmpy42X0EUtgkd6',
  'name': 'Oude Maasweg',
  'artists': ['The Amazing Stroopwafels'],
  'similar': 0.44175341725349426},
 {'id': '1Qw4FBX30bGdutJ19HBD7T',
  'name': 'Lovefool (feat. Pia Mia)',
  'artists': ['Pia Mia', 'twocolors'],
  'similar': 0.4429745376110077},
 {'id': '38mpo90GPSUHgy2dUVvpZO',
  'name': 'Maombi',
  'artists': ['Nadia Mukami'],
  'similar': 0.42666658759117126},
 {'id': '0H351Cn5b1uS4E4Cg1Jxb5',
  'name': 'La Carta',
  'artists': ['NAFTA'],
  'similar': 0.43813273310661316},
 {'id': '3p0GBN1o2XyWsI7l6o3iGB',
  'name': 'It\'s Him (The Majestic Tale) [From "Doctor Who - The Day of The Doctor"]',


In [20]:
suggest_from("air on G")

[{'id': '0mD1a7haZKdX9I0oPywrMb',
  'name': 'Air on a G String',
  'artists': ['HAUSER',
   'Johann Sebastian Bach',
   'London Symphony Orchestra',
   'Robert Ziegler'],
  'count': 45},
 {'id': '2eIhmj1inaH2P4SYpgYZ78',
  'name': 'Aynı Sokaklarda',
  'artists': ['Şanışer'],
  'similar': 0.4385406970977783},
 {'id': '2mhju5F4Hba8xX2udLVSaX',
  'name': 'Trilogy',
  'artists': ['Silent Planet'],
  'similar': 0.4212125837802887},
 {'id': '0JUaqijY6i3hBzOdHGMP9j',
  'name': 'Real Gone',
  'artists': ['Sheryl Crow'],
  'similar': 0.4512108266353607},
 {'id': '4fgJAlKZFuc8Nlq30fq8Tq',
  'name': 'Eclypz',
  'artists': ['Owl Vision'],
  'similar': 0.4628591537475586},
 {'id': '6eXn9NxCovcNIQ8wXIIZv0',
  'name': 'Jauh',
  'artists': ['Nadya Fatira'],
  'similar': 0.43133726716041565},
 {'id': '03KPztknFhsbbSbA0CSjbf',
  'name': 'Go Tell It on the Mountain (Instrumental Version)',
  'artists': ['Ivy Ravenwood'],
  'similar': 0.41745278239250183},
 {'id': '1rU1OYNaz6h4BXKP0UB2A2',
  'name': 'Hill

In [15]:
# conn.close()